# 1.解析対象ポリゴンに土壌実測データ結合

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
gpd_data = gpd.read_file('/workspaces/akita_daizu/data/474+120圃場.geojson')

In [3]:
soil_data = pd.read_excel('/workspaces/akita_daizu/data/0124修正_土壌データ+収量.xlsx', header=0)

In [4]:
# gpdの結合用うの行の削除と作成
gpd_data['作付け開始年'] = ''
gpd_data['圃場番号'] = gpd_data['uniqueID']
gpd_data['排水性の良否'] = gpd_data['pred_labels']
gpd_data['EVI20902mean'] = gpd_data['EVI2mean']

# gpdの必要な行だけ抽出して結合する ほしい行は以下の4行
selected_gpd = gpd_data[['圃場番号','排水性の良否','作付年_y', 'H23以前_y','播種月日','作付け開始年','EVI20902mean','EVI20903mean','geometry']]

In [5]:
#　この作付け年とH23以前を一つの列に。　まず作付け年が空ならH23以前を入れ込むようにFor
for index, row in selected_gpd[['作付年_y', 'H23以前_y','作付け開始年']].iterrows():
    if row['H23以前_y'] is not None :
        selected_gpd['作付け開始年'][index] =row['H23以前_y']
    elif  row['作付年_y'] is not None :
        selected_gpd['作付け開始年'][index] = row['作付年_y']

# それぞれの和暦を西暦に変更
# H23 2011 H24 2012 H25 2013 R1 2019
selected_gpd['作付け開始年'] = selected_gpd['作付け開始年'].str.replace(r'Ｒ3～',r'2021',)
selected_gpd['作付け開始年'] = selected_gpd['作付け開始年'].str.replace(r'Ｒ2～',r'2020',)
selected_gpd['作付け開始年'] = selected_gpd['作付け開始年'].str.replace(r'Ｒ1～',r'2019',)
selected_gpd['作付け開始年'] = selected_gpd['作付け開始年'].str.replace(r'Ｈ30～',r'2017',)
selected_gpd['作付け開始年'] = selected_gpd['作付け開始年'].str.replace(r'Ｈ28～',r'2016',)
selected_gpd['作付け開始年'] = selected_gpd['作付け開始年'].str.replace(r'Ｈ27～',r'2015',)
selected_gpd['作付け開始年'] = selected_gpd['作付け開始年'].str.replace(r'Ｈ26～',r'2014',)
selected_gpd['作付け開始年'] = selected_gpd['作付け開始年'].str.replace(r'Ｈ25～',r'2013',)
selected_gpd['作付け開始年'] = selected_gpd['作付け開始年'].str.replace(r'Ｈ24～',r'2012',)
selected_gpd['作付け開始年'] = selected_gpd['作付け開始年'].str.replace(r'☑',r'2011',)
selected_gpd['作付け開始年'].unique()

target_selected_gpd = selected_gpd

/tmp/ipykernel_17844/436268765.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_gpd['作付け開始年'][index] =row['H23以前_y']
/tmp/ipykernel_17844/436268765.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_gpd['作付け開始年'][index] = row['作付年_y']
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(Ge

In [6]:
#まず　GPDとSOILデータでuniqueIDをキーにして全部結合する
merged_pd = pd.merge(soil_data, target_selected_gpd, on=['圃場番号'], how='inner')

In [53]:
# 有効数字3桁で丸め
merged_pd = merged_pd.round(3)

In [72]:
# heatmap作成のため不要な行の削除 土壌硬度周り
drop_columns = merged_pd.filter(like='土壌硬度', axis=1).columns.values
droped_soil_data = merged_pd.drop(drop_columns, axis = 1)
# 現場測定値も不要
drop_columns = ['播種月日_x','現場測定_EC(㎳／㎝)', '現場測定_PH','生育状況','最大圧の深さの平均', '最大圧力の平均','調査日', '現物重量ｋｇ','作付年_y','H23以前_y', '計量時水分％', '圃場面積', '住所','EVI20903mean', '反収現物重量kg/10a',
       '15%時の重量']
mymaps_data = droped_soil_data.drop(drop_columns, axis = 1)

In [73]:
mymaps_data.columns

Index(['圃場番号', 'SoilName', '15%反収現物重量kg/10a', '実験室測定_EC(㎳／㎝)', '実験室測定_pH',
       'アンモニア態窒素(mgN/100g)', '硝酸態窒素(mgN/100g)', '可給隊窒素(mgN/100g)',
       '有効態リン酸(mgP2O5/100g)', 'TC(%)', '腐食(%)', 'TN(%)', '排水性の良否',
       '播種月日_y', '作付け開始年', 'EVI20902mean', 'geometry'],
      dtype='object')

このデータの連続値をカテゴリにしないといけない　"最大値と最小値を5等分"　で作成

In [74]:
target_columns = ['15%反収現物重量kg/10a', '実験室測定_EC(㎳／㎝)', '実験室測定_pH',
       'アンモニア態窒素(mgN/100g)', '硝酸態窒素(mgN/100g)', '可給隊窒素(mgN/100g)',
       '有効態リン酸(mgP2O5/100g)', 'TC(%)', '腐食(%)', 'TN(%)', 'EVI20902mean']
qcut_df = mymaps_data
for x,i in enumerate(target_columns):
    qcut_df[i] = pd.qcut(mymaps_data[i], 5)
    # カテゴリデータ型になっていると出力できないのでstrにする
    qcut_df[i] = qcut_df[i].astype(str)

In [77]:
from shapely import wkt
qcut_df['geometry'] = gpd.GeoSeries.from_wkt(qcut_df['geometry'])

In [83]:
gdf_mymaps_data = qcut_df.set_crs('EPSG:6678')

In [85]:
# 見えるかようのポリゴンの抽出
gdf_mymaps_data = gpd.GeoDataFrame(qcut_df, geometry='geometry')
gdf_mymaps_data.to_file('/workspaces/akita_daizu/data/mymaps.geojson', driver="GeoJSON")  
